#### ***(1) 상권 경쟁 및 밀도 변수 (시군구 기준)***

| 변수명 | 생성 방식 |
|------|----------|
| 시군구 요리주점 수 | 시군구 기준 요리주점 점포 count |
| 시군구 면적 | 외부 시군구 행정면적 데이터 병합 |
| 요리주점 밀도 | 시군구 요리주점 수 / 시군구 면적 |
| 프랜차이즈 비율 | 시군구 내 프랜차이즈 요리주점 수 / 전체 요리주점 수 |

### ***프랜차이즈 비율을 아래처럼 재정의***
- 시군구 내 금별맥주 매장 수 / (금별 + 역전 매장 수)
    - “프랜차이즈 내부 점유율”
    - 상권 전체 비율 아님
- 변수명도 바꿔야 함
- 프랜차이즈 비율 X
- 브랜드 점유율 / 프랜차이즈 내부 비중

### ***3.1_시군구_상권경쟁_수요변수***
1. 시군구별 매장 수 (경쟁)
    - 의미: 경쟁 규모 (공급량)
2. 시군구 평균 월간 검색량 (수요)
    - 의미: 수요 대리지표
3. 시군구 평균 블로그 누적 발행량 (관심)
    - 의미: 관심도, 콘텐츠 반응

***본 분석은 시군구 단위에서 매장 수를 경쟁 강도의 대표 지표로, 검색량과 블로그 발행량을 수요 및 관심도의 대리지표로 사용함***

In [1]:
# 1. 라이브러리 로드
import pandas as pd
import numpy as np

In [13]:
# 1. 프랜차이즈 요리주점 데이터 로드
# =================================================
FILES = {
    "금별맥주": r"F:\데이터분석\정제\금별맥주_정제완료.csv",
    "역전할머니": r"F:\데이터분석\정제\역전할머니_정제완료.csv"
}

dfs = {}
for brand, path in FILES.items():
    df = pd.read_csv(path)
    df["브랜드"] = brand
    dfs[brand] = df
    print(f"{brand} 로드 완료:", df.shape)

금별맥주 로드 완료: (103, 35)
역전할머니 로드 완료: (327, 35)


In [14]:
# 2. 공통 컬럼 검증
# =================================================
REQUIRED_COLS = [
    "시군구명",
    "상호명",
    "월간 검색량",
    "블로그누적발행"
]

for brand, df in dfs.items():
    missing = [c for c in REQUIRED_COLS if c not in df.columns]
    if missing:
        raise ValueError(f"{brand} 필수 컬럼 누락: {missing}")

In [15]:
# 3. 결측치 처리
# =================================================
for df in dfs.values():
    df["월간 검색량"] = df["월간 검색량"].fillna(0)
    df["블로그누적발행"] = df["블로그누적발행"].fillna(0)

In [16]:
# 4. 시군구 단위 요약 (경쟁·수요)
# =================================================
summary_list = []

for brand, df in dfs.items():
    tmp = (
        df.groupby("시군구명")
        .agg(
            시군구_요리주점_수=("상호명", "count"),
            평균월간검색량=("월간 검색량", "mean"),
            평균블로그발행량=("블로그누적발행", "mean")
        )
        .reset_index()
    )
    tmp["브랜드"] = brand
    summary_list.append(tmp)

시군구_요약 = pd.concat(summary_list, ignore_index=True)

In [17]:
# 5. 프랜차이즈 요리주점 통합 (금별 + 역전)
# =================================================
프랜차이즈_df = pd.concat(dfs.values(), ignore_index=True)

# 혹시 시군구명이 인덱스로 들어가 있을 경우 대비
if "시군구명" not in 프랜차이즈_df.columns:
    프랜차이즈_df = 프랜차이즈_df.reset_index()

프랜차이즈_시군구 = (
    프랜차이즈_df
    .groupby("시군구명")
    .size()
    .reset_index(name="시군구_프랜차이즈_요리주점_수")
)

In [20]:
# 6. 시군구 면적 데이터 로드 (서울 + 경기)
# 4. 분석 프레임워크 Step 1. 데이터전처리 파일
# =================================================
면적_서울 = pd.read_csv(r"F:\데이터분석\정제\4.2_시군구_서울_면적.csv")
면적_경기 = pd.read_csv(r"F:\데이터분석\정제\4.2_시군구_경기_면적.csv")

df_면적 = pd.concat([면적_서울, 면적_경기], ignore_index=True)

# 인덱스 방어
if "시군구명" not in df_면적.columns:
    df_면적 = df_면적.reset_index()

In [ ]:
# 병합 전 확인
# 병합 키(시군구명)가 모든 데이터프레임에 컬럼으로 존재하는지 사전 점검
print("시군구_요약 columns:", 시군구_요약.columns)
print("프랜차이즈_시군구 columns:", 프랜차이즈_시군구.columns)
print("df_면적 columns:", df_면적.columns)

시군구_요약 columns: Index(['시군구명', '시군구_요리주점_수', '평균월간검색량', '평균블로그발행량', '브랜드'], dtype='object')
프랜차이즈_시군구 columns: Index(['시군구명', '시군구_프랜차이즈_요리주점_수'], dtype='object')
df_면적 columns: Index(['index', '시군구', '면적_km2'], dtype='object')


In [26]:
# =================================================
# df_면적 컬럼 정리 (필수)
# =================================================
df_면적 = (
    df_면적
    .rename(columns={
        "시군구": "시군구명",
        "면적_km2": "시군구면적"
    })
    .drop(columns=["index"], errors="ignore")
)

print(df_면적.columns)

Index(['시군구명', '시군구면적'], dtype='object')


In [25]:
# 7. 병합
# =================================================
시군구_요약 = (
    시군구_요약
    .merge(프랜차이즈_시군구, on="시군구명", how="left")
    .merge(df_면적, on="시군구명", how="left")
)

In [27]:
# 8. 밀도·경쟁 지표 생성
# =================================================
시군구_요약["요리주점_밀도"] = (
    시군구_요약["시군구_프랜차이즈_요리주점_수"] /
    시군구_요약["시군구면적"]
)

시군구_요약["경쟁강도지수"] = 시군구_요약["시군구_요리주점_수"]

시군구_요약["수요대비경쟁지수"] = (
    시군구_요약["시군구_요리주점_수"] /
    (시군구_요약["평균월간검색량"] + 1)
)

In [30]:
# 9. 결과 확인
# =================================================
시군구_밀도 = (
    시군구_요약
    .sort_values(by="요리주점_밀도", ascending=False)
    .head(10)
)

시군구_밀도.head(10)

,시군구명,시군구_요리주점_수,평균월간검색량,평균블로그발행량,브랜드,시군구_프랜차이즈_요리주점_수,시군구면적,요리주점_밀도,경쟁강도지수,수요대비경쟁지수
106,중구,4,20.00,269.750000,역전할머니,5,19.92,0.251004,4,0.190476
41,중구,1,20.00,32.000000,금별맥주,5,19.92,0.251004,1,0.047619
8,구로구,3,20.00,56.000000,금별맥주,10,40.24,0.248509,3,0.142857
58,구로구,7,20.00,161.571429,역전할머니,10,40.24,0.248509,7,0.333333
68,동작구,8,21.25,145.501250,역전할머니,8,32.70,0.244648,8,0.359551
32,영등포구,3,20.00,28.666667,금별맥주,12,49.10,0.244399,3,0.142857
95,영등포구,9,20.00,133.555556,역전할머니,12,49.10,0.244399,9,0.428571
61,금천구,6,20.00,187.000000,역전할머니,6,26.04,0.230415,6,0.285714
57,광진구,6,20.00,203.500000,역전할머니,7,34.12,0.205158,6,0.285714
7,광진구,1,20.00,118.000000,금별맥주,7,34.12,0.205158,1,0.047619


In [31]:
# 10. 저장
# =================================================
OUTPUT_PATH = r"F:\데이터분석\정제\3.1_시군구_상권경쟁_수요_밀도변수.csv"

시군구_요약.to_csv(
    OUTPUT_PATH,
    index=False,
    encoding="utf-8-sig"
)